参考文档教程：
https://www.langchain.com.cn/docs/integrations/text_embedding/ollama/

在本教程中，我们将学习如何利用本地ollama的向量模型进行嵌入；以及利用内存数据库进行向量检索。同时附上了如何自定义向量嵌入的教程。

#安装依赖：

## 安装uv

pip install uv -i https://pypi.tuna.tsinghua.edu.cn/simple/

## 安装依赖包

uv pip install  langgraph "langchain[openai]" langchain-community   langchain-ollama -i https://pypi.tuna.tsinghua.edu.cn/simple/






# 使用 Ollama 进行嵌入
"""
现在我们有了拆分的文档，我们可以将它们索引到用于语义搜索的向量存储中。
"""

拉取向量的命令：
ollama pull bge-m3:latest

"""



In [ ]:
# 1. 创建向量嵌入模型



from langchain_ollama import OllamaEmbeddings

local_embeddings = OllamaEmbeddings(
    model="bge-m3:latest",
    base_url="192.168.3.84:11434",
)

print(local_embeddings)




model='bge-m3:latest' base_url='192.168.3.136:11434' client_kwargs={} async_client_kwargs={} sync_client_kwargs={} mirostat=None mirostat_eta=None mirostat_tau=None num_ctx=None num_gpu=None keep_alive=None num_thread=None repeat_last_n=None repeat_penalty=None temperature=None stop=None tfs_z=None top_k=None top_p=None


In [37]:
# 2 单个嵌入示例

text1 = "LangChain is the framework for building context-aware reasoning applications"
single_vector = local_embeddings.embed_query(text1)
print(str(single_vector)[:100])  # Show the first 100 characters of the vector

[-0.01606138, -0.011700507, 0.0042311954, -0.011310199, -0.017729746, -0.051959366, 0.019131344, 0.0


In [38]:
#3 嵌入多个文本
text2 = (
    "LangGraph is a library for building stateful, multi-actor applications with LLMs"
)
two_vectors = local_embeddings.embed_documents([text, text2])
for vector in two_vectors:
    print(str(vector)[:100])  # Show the first 100 characters of the vector

[-0.01606138, -0.011700507, 0.0042311954, -0.011310199, -0.017729746, -0.051959366, 0.019131344, 0.0
[-0.07276119, 1.7207447e-05, -0.001866579, -0.03082587, -0.018602427, -0.008962815, -0.019794874, -0


In [39]:
# 4 使用内存数据库进行向量检索
# https://python.langchain.com/v0.2/api_reference/core/vectorstores/langchain_core.vectorstores.in_memory.InMemoryVectorStore.html


from langchain_core.vectorstores import InMemoryVectorStore
text1 = "LangChain is the framework for building context-aware reasoning applications"
text2 = "LangGraph is a library for building stateful, multi-actor applications with LLMs"
text3 = "我是AI小新，你的智能助手，很高兴为你服务。"
text4 = "我会根据你的需求，提供各种帮助和服务。"

vectorstore = InMemoryVectorStore.from_texts(
    [text1,text2,text3,text4],
    embedding=local_embeddings,
)

results = vectorstore.similarity_search_with_score(
    query="你是谁", k=2,similarity_threshold=0.6
)
for doc, score in results:
    print(f"* [SIM={score:3f}] {doc.page_content} [{doc.metadata}]")







* [SIM=0.502678] 我是AI小新，你的智能助手，很高兴为你服务。 [{}]
* [SIM=0.433895] 我会根据你的需求，提供各种帮助和服务。 [{}]


In [40]:
# Use as Retriever:

retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k":2, "fetch_k": 2, "lambda_mult": 0.5},
)
retriever.invoke("你是谁?")

for doc, score in results:
    print(f"* [SIM={score:3f}] {doc.page_content} [{doc.metadata}]")

* [SIM=0.502678] 我是AI小新，你的智能助手，很高兴为你服务。 [{}]
* [SIM=0.433895] 我会根据你的需求，提供各种帮助和服务。 [{}]


In [41]:
# 如果内置的嵌入方法满足不了你，你可以自定义嵌入的方法；

"""
自定义嵌入

参考文档：https://python.langchain.ac.cn/docs/how_to/custom_embeddings/


自定义嵌入
LangChain 与许多 第三方嵌入模型 集成。在本指南中，我们将向您展示如何创建自定义 Embedding 类，以防内置的类尚不存在。嵌入在自然语言处理应用中至关重要，因为它们将文本转换为算法可以理解的数字形式，从而实现广泛的应用，例如相似性搜索、文本分类和聚类。

使用标准的 Embeddings 接口实现嵌入，将允许您的嵌入在现有的 LangChain 抽象中使用（例如，作为 VectorStore 的驱动嵌入，或使用 CacheBackedEmbeddings 进行缓存）。

"""

#自定义嵌入 的使用 示例

from typing import List

from langchain_core.embeddings import Embeddings


class ParrotLinkEmbeddings(Embeddings):
    """ParrotLink embedding model integration.

    # TODO: Populate with relevant params.
    Key init args — completion params:
        model: str
            Name of ParrotLink model to use.

    See full list of supported init args and their descriptions in the params section.

    # TODO: Replace with relevant init params.
    Instantiate:
        .. code-block:: python

            from langchain_parrot_link import ParrotLinkEmbeddings

            embed = ParrotLinkEmbeddings(
                model="...",
                # api_key="...",
                # other params...
            )

    Embed single text:
        .. code-block:: python

            input_text = "The meaning of life is 42"
            embed.embed_query(input_text)

        .. code-block:: python

            # TODO: Example output.

    # TODO: Delete if token-level streaming isn't supported.
    Embed multiple text:
        .. code-block:: python

             input_texts = ["Document 1...", "Document 2..."]
            embed.embed_documents(input_texts)

        .. code-block:: python

            # TODO: Example output.

    # TODO: Delete if native async isn't supported.
    Async:
        .. code-block:: python

            await embed.aembed_query(input_text)

            # multiple:
            # await embed.aembed_documents(input_texts)

        .. code-block:: python

            # TODO: Example output.

    """

    def __init__(self, model: str):
        self.model = model

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """Embed search docs."""
        return [[0.5, 0.6, 0.7] for _ in texts]

    def embed_query(self, text: str) -> List[float]:
        """Embed query text."""
        return self.embed_documents([text])[0]

    # optional: add custom async implementations here
    # you can also delete these, and the base class will
    # use the default implementation, which calls the sync
    # version in an async executor:

    # async def aembed_documents(self, texts: List[str]) -> List[List[float]]:
    #     """Asynchronous Embed search docs."""
    #     ...

    # async def aembed_query(self, text: str) -> List[float]:
    #     """Asynchronous Embed query text."""
    #     ...



# 自定义嵌入的示例

custom_embeddings = ParrotLinkEmbeddings("自定义嵌入模型")
# 多个嵌入的示例
print(custom_embeddings.embed_documents(["我是AI小新", "我是自定义嵌入示例"]))

# 单个嵌入的示例
print(custom_embeddings.embed_query("你是谁"))


[[0.5, 0.6, 0.7], [0.5, 0.6, 0.7]]
[0.5, 0.6, 0.7]
